In [1]:
import pandas as pd

In [47]:
train = pd.read_csv('train.csv')

In [48]:
test = pd.read_csv('test.csv')

In [49]:
y = train['target']

In [50]:
train = train.drop('target', axis=1)

In [51]:
   
def preproc_data(df_input):
    df_output = df_input.copy()
    
    df_output = df_output.drop('_id', axis=1)
    


    month = list(set(df_output['month']))
    df_output['month'] = df_output.month.apply(lambda r: month.index(r))   
    


    df_output['duration_interrupted'] = df_output['duration'].apply(lambda d: 0 if d < 30 else d)
    
    df_output['nr.employed'] = df_output['nr.employed'].apply(lambda x: 1 if x <= 5087.65 else 0)
    
    df_output['default'] = df_output['default'].apply(lambda x: 0 if x == 'no' else 1)
    
    df_output['age_class'] = df_output['age'].apply(lambda x: 1 if (x >= 25 and x <  50) else 0)
    
    df_output = df_output.drop(['month', 'day_of_week', 'contact', 'pdays'], axis = 1)
    
    df_output['duration_class_my'] = df_output['duration'].apply(lambda x: 1 if (x >= 197 and x <= 700) else 0)

    df_output = pd.get_dummies(df_output, columns=[
         'marital',
         'education',
         'default',
         'housing',
         'loan',
       #  'contact',
         'poutcome' ,
        'job'
        #'month'
        
        
    ])
    
    return df_output

In [52]:
# train_work = preproc_data(train)
# test_work = preproc_data(test)

train_work = preproc_data(train)
test_work = preproc_data(test)

In [32]:
train_work.head(10)

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed
0,54,87,1,999,0,1.4,93.444,-36.1,4.963,5228.1,...,0,0,0,0,0,0,0,0,1,0
1,36,291,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,0,0
2,53,182,1,999,0,1.4,93.444,-36.1,4.965,5228.1,...,0,0,0,0,0,0,1,0,0,0
3,34,180,2,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,0,0,0,1
4,29,6,1,999,0,-1.7,94.027,-38.3,0.890,4991.6,...,0,0,0,0,0,1,0,0,0,0
5,56,64,2,999,0,1.4,94.465,-41.8,4.961,5228.1,...,0,0,0,0,0,0,0,1,0,0
6,33,312,2,999,0,-1.8,93.876,-40.0,0.685,5008.7,...,0,1,0,0,0,0,1,0,0,0
7,36,20,1,999,1,-0.1,93.200,-42.0,4.120,5195.8,...,0,0,1,0,0,0,0,0,0,1
8,70,585,1,6,3,-1.1,94.601,-49.5,0.987,4963.6,...,0,0,0,1,0,0,0,1,0,0
9,32,196,2,999,1,-1.8,92.893,-46.2,1.327,5099.1,...,0,1,0,0,0,0,0,1,0,0


In [13]:
from sklearn.model_selection import train_test_split

In [53]:
X_train, X_test, y_train, y_test = train_test_split(train_work, y, test_size=0.33, random_state=42)

In [15]:
from sklearn.tree import DecisionTreeClassifier

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [17]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

# classifiers = {
#     'tree': DecisionTreeClassifier(max_depth=5), 
#     'logistic': LogisticRegression(),
#     'forest': RandomForestClassifier(),
#     'kn': KNeighborsClassifier(n_neighbors=10)
# }

In [59]:
import numpy as np
dtc = DecisionTreeClassifier(max_depth=10)
dtc.fit(X_train, y_train)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

lr = LogisticRegression()
# lr = LogisticRegression(n_jobs=-1, class_weight='balanced', solver='newton-cg', multi_class='multinomial')
lr.fit(X_train, y_train)

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

print("Feature ranking:")

for f in range(X_train.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, train_work.columns[indices[f]], importances[indices[f]]))

Feature ranking:
1. feature duration_interrupted (0.176180)
2. feature duration (0.164588)
3. feature euribor3m (0.096635)
4. feature age (0.068161)
5. feature nr.employed (0.067213)
6. feature cons.conf.idx (0.041472)
7. feature cons.price.idx (0.040996)
8. feature emp.var.rate (0.032348)
9. feature campaign (0.031865)
10. feature poutcome_success (0.028867)
11. feature duration_class_my (0.017934)
12. feature previous (0.014663)
13. feature housing_no (0.011807)
14. feature housing_yes (0.011791)
15. feature education_university.degree (0.011754)
16. feature job_admin. (0.011148)
17. feature marital_married (0.010757)
18. feature age_class (0.009862)
19. feature education_high.school (0.009858)
20. feature marital_single (0.009785)
21. feature poutcome_failure (0.008497)
22. feature loan_no (0.008464)
23. feature job_technician (0.008300)
24. feature loan_yes (0.008157)
25. feature job_blue-collar (0.007779)
26. feature education_professional.course (0.007566)
27. feature poutcome_no

In [16]:
roc_auc_score(y_test, dtc.predict_proba(X_test)[:,1])

0.8462023101940114

In [39]:
roc_auc_score(y_test, lr.predict_proba(X_test)[:,1])

0.9288337632238048

In [18]:
roc_auc_score(y_test, knn.predict_proba(X_test)[:,1])

0.7834122088893875

In [19]:
from sklearn.metrics import log_loss
log_loss(y_test, dtc.predict_proba(X_test)[:,1])

1.1524042093425304

In [40]:
log_loss(y_test, lr.predict_proba(X_test)[:,1])

0.21230478884201134

In [21]:
log_loss(y_test, knn.predict_proba(X_test)[:,1])

1.5940843728644436

In [22]:
train_work_tmp = train_work.copy()
train_work_tmp['dtc'] = dtc.predict_proba(train_work)[:,1]


train_work_tmp['rf'] = rf.predict_proba(train_work)[:,1]
train_work = train_work_tmp

test_work_tmp = test_work.copy()
test_work_tmp['dtc'] = dtc.predict_proba(test_work)[:,1]


test_work_tmp['rf'] = rf.predict_proba(test_work)[:,1]
test_work = test_work_tmp

In [55]:
X_train, X_test, y_train, y_test = train_test_split(train_work, y, test_size=0.33, random_state=42)
lr.fit(X_train, y_train)
roc_auc_score(y_test, lr.predict_proba(X_test)[:,1])

0.9288337632238048

In [60]:
lr.fit(train_work, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [61]:
result_predict = pd.DataFrame(data={'_id': test['_id'], 'target': lr.predict_proba(test_work)[:,1]})

In [62]:
result_predict_test = pd.DataFrame(data={'_id': test['_id'], 'target': lr.predict(test_work)})

In [63]:
result_predict_test[result_predict_test['target'] == 1]['target'].agg('count')

1130

In [ ]:
# del result_predict

In [64]:
result_predict.to_csv('submit_lr.csv', index=False)